<a href="https://colab.research.google.com/github/evcereti/Aprendiendo_ML_y_DS/blob/main/regresion_polinomial_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Regresión Polinomial

##Importando las librerias a utilizar

In [16]:
import pandas as pd

##Importando el Dataset

In [17]:
ds = pd.read_csv('car data.csv')

##Análisis Exploratorio de los Datos y limpieza del Dataset

Como siempre, partimos de entender que tipo de dataset nos fue entregado.

In [18]:
ds.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


Tenemos 9 columnas, de las cuales 4 son categoricas y 5 son numéricas. En tanto que este es un ejercicio en regresión polinomial, decidimos que la variable continua a predecir debería ser Selling_Price.

Lo siguiente es analizar la validez de los datos.

In [19]:
ds.shape

(301, 9)

In [20]:
ds.isnull().sum()

,0
Car_Name,0
Year,0
Selling_Price,0
Present_Price,0
Kms_Driven,0
Fuel_Type,0
Seller_Type,0
Transmission,0
Owner,0


In [21]:
ds[ds.duplicated(keep=False)]

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
15,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
17,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
51,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0
93,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0


In [22]:
ds = ds.drop_duplicates(keep='first')

In [25]:
ds.shape

(299, 9)

El dataset tiene original tenía 301 puntos muestrales, con 2 duplicados y sin datos faltantes. Después de eliminar los duplicados, nos quedamos con 299 elementos.

In [8]:
ds['Car_Name'].nunique()

98

El dataset tiene 98 modelos de auto únicos. Este número es muy grande en comparación al tamaño del dataset.

Revisamos si es posible disminuir este numero mediante algún tipo de limpieza.

In [10]:
ds['Car_Name'].values

array(['ritz', 'sx4', 'ciaz', 'wagon r', 'swift', 'vitara brezza', 'ciaz',
       's cross', 'ciaz', 'ciaz', 'alto 800', 'ciaz', 'ciaz', 'ertiga',
       'dzire', 'ertiga', 'ertiga', 'ertiga', 'wagon r', 'sx4',
       'alto k10', 'ignis', 'sx4', 'alto k10', 'wagon r', 'swift',
       'swift', 'swift', 'alto k10', 'ciaz', 'ritz', 'ritz', 'swift',
       'ertiga', 'dzire', 'sx4', 'dzire', '800', 'alto k10', 'sx4',
       'baleno', 'alto k10', 'sx4', 'dzire', 'omni', 'ciaz', 'ritz',
       'wagon r', 'ertiga', 'ciaz', 'fortuner', 'fortuner', 'innova',
       'fortuner', 'innova', 'corolla altis', 'etios cross',
       'corolla altis', 'etios g', 'fortuner', 'corolla altis',
       'etios cross', 'fortuner', 'fortuner', 'fortuner', 'etios liva',
       'innova', 'fortuner', 'corolla altis', 'corolla altis',
       'etios liva', 'corolla altis', 'corolla altis', 'etios liva',
       'etios cross', 'etios g', 'corolla altis', 'corolla',
       'corolla altis', 'fortuner', 'corolla altis', 'e

Aquí vemos que Car_Name contiene Marca y modelo, de forma que debería ser posible separar esta información en dos columnas diferentes. Sin un diccionario de marcas y modelos es muy dificil lograr este objetivo, así que lo mejor es eliminar por completo la variable.

In [11]:
ds = ds.drop('Car_Name', axis=1)

In [12]:
ds.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
